# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import copy
import datetime
from rentcast_dapi import *

%load_ext autoreload
%autoreload 2

# RentCast Overview

In [ ]:
# API docs - https://developers.rentcast.io/reference/introduction
# Pricing - https://www.rentcast.io/api#api-pricing
# Account Dashboard - https://app.rentcast.io/app/api

rent_cast_api_key = 'fcc15170bef044e896b098360c6fe570'

# Use simple API wrapper

In [ ]:
rcd = RentCastData(city='Littleton', state='CO', offset_lim=1000, api_key=rent_cast_api_key)
print(rcd.data_processed.shape)
rcd.data_processed.head()

In [ ]:
rcd.data_processed.head()

In [ ]:
rcd = RentCastData(city='Woodside', state='CA', offset_lim=1000, api_key=rent_cast_api_key)
print(rcd.data_processed.shape)
rcd.data_processed.head()
rcd.save_to_db(db_folder='./Data')

# Show what data already exists in the SQL database

In [ ]:
dataset_path = './Data/CA'

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(dataset_path + '.db')

# Query to get all table names
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Use pandas to execute the query and fetch the table names into a DataFrame
tables_df = pd.read_sql_query(query, conn)

# Extract the table names from the DataFrame
table_names = tables_df['name'].tolist()
table_names.sort()

# Close the connection
conn.close()

# Display the list of tables
print(table_names)

In [ ]:
rcd.querystring['city']

# Read data in from a SQL database

In [ ]:
rcd = RentCastData.open_db(db_folder='./Data', city='Redwood City', state='CA')
# rcd = RentCastData.open_db(db_folder='./Data', city='Littleton', state='CO')

In [ ]:
rcd = RentCastData.open_db(db_folder='./Data', city='Woodside', state='CA')

# Simple Plots

In [ ]:
df = rcd.parse_sale_date2(rcd.data_processed)

check1 = df['squareFootage'] > 1000
check2 = df['squareFootage'] < 3000
df_temp  = df[check1 & check2]
# df_temp = df

x = df_temp['months'].values / 12
y = df_temp['price_per_sqft'].values

# Avererages
grouped_df = df_temp.groupby('months')['price_per_sqft'].mean().reset_index()
x_avg = grouped_df['months'].values / 12
y_avg = grouped_df['price_per_sqft'].values

# y_average = df.average
plt.figure(figsize=(10, 4))

# plt.plot(x, y, lw=0, ms=2, marker='o', alpha=0.2)
# plt.plot(x_avg, y_avg, lw=1, ms=0, marker='o', color='black', alpha=0.5)
# plt.ylim([-10, 2000])

plt.semilogy(x, y, lw=0, ms=2, marker='o', alpha=0.1)
plt.semilogy(x_avg, y_avg, lw=1, ms=0, marker='o', color='black', alpha=0.5)
plt.ylim([50, 2000])


# plt.xlim([40, 60])

# x labels
int_labels = np.arange(0, 60, 2.5)
n = len(int_labels)
lsd = df['lastSaleDate'].values[0]
dt = datetime.datetime.strptime(lsd, '%Y-%m-%dT%H:%M:%S.%fZ')
dt_labels = [rcd.add_months(dt, x * 30) for x in range(n)]
dt_labels = [dt_label.strftime('%m/%Y') for dt_label in dt_labels]
plt.xticks(int_labels, dt_labels, rotation=45)
plt.yticks([10, 30, 100, 300, 1000], [10, 30, 100, 300, 1000])

plt.title('Price per SQFT - Redwood City')
plt.ylabel('Price per SQFT ($)')
plt.xlabel('Time')
plt.grid(True)
# plt.xlim([40, 55])

In [ ]:
df = rcd.parse_sale_date2(rcd.data_processed)

check1 = df['squareFootage'] > 1000
check2 = df['squareFootage'] < 2000
df_temp  = df[check1 & check2]
# df_temp = df

x = df_temp['months'].values / 12
y = df_temp['price_per_sqft'].values

# Avererages
grouped_df = df_temp.groupby('months')['price_per_sqft'].mean().reset_index()
x_avg = grouped_df['months'].values / 12
y_avg = grouped_df['price_per_sqft'].values

# y_average = df.average
plt.figure(figsize=(10, 4))

plt.plot(x, y, lw=0, ms=2, marker='o', alpha=0.2)
plt.plot(x_avg, y_avg, lw=1, ms=0, marker='o', color='black', alpha=0.5)
plt.ylim([-10, 2000])

# plt.semilogy(x, y, lw=0, ms=2, marker='o', alpha=0.2)
# plt.semilogy(x_avg, y_avg, lw=1, ms=0, marker='o', color='black', alpha=0.5)
# plt.ylim([50, 2000])


# plt.xlim([40, 60])

# x labels
int_labels = np.arange(0, 60, 2.5)
n = len(int_labels)
lsd = df['lastSaleDate'].values[0]
dt = datetime.datetime.strptime(lsd, '%Y-%m-%dT%H:%M:%S.%fZ')
dt_labels = [rcd.add_months(dt, x * 30) for x in range(n)]
dt_labels = [dt_label.strftime('%m/%Y') for dt_label in dt_labels]
plt.xticks(int_labels, dt_labels, rotation=45)

plt.grid(True)

# Plot several cities

In [ ]:
dataset_path = './Data/CA'

# Connect to the SQLite database
conn = sqlite3.connect(dataset_path + '.db')

# Query to get all table names
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Use pandas to execute the query and fetch the table names into a DataFrame
tables_df = pd.read_sql_query(query, conn)

# Extract the table names from the DataFrame
table_names = tables_df['name'].tolist()
table_names.sort()

# Close the connection
conn.close()

# Display the list of tables
print(table_names)

In [ ]:
def sos_filter(y, fs, fc, highlow='low', order=1):
    y = np.array(y)
    length = len(y)
    y_temp = np.append(y[::-1], np.append(y, y[::-1]))
    sos = signal.butter(fs=fs, Wn=fc, N=order, output='sos', btype=highlow)
    y_sos = signal.sosfiltfilt(sos, y_temp)
    y_sos = y_sos[length:length*2]

    return y_sos

In [ ]:
cities1 = ['atherton_ca', 'los_altos_ca', 'menlo_park_ca', 'redwood_city_ca', 'woodside_ca']
cities2 = ['belmont_ca', 'mountain_view_ca', 'palo_alto_ca', 'redwood_city_ca', 'san_carlos_ca', 'san_mateo_ca']

In [ ]:
filter = True

plt.figure(figsize=(10, 4))

for index, table_name in enumerate(table_names):

    table_name_split = table_name.split('_')
    state = table_name_split[-1]
    city = " ".join(table_name_split[:-1])
    label = '%s - %s' % (city.title(), state.upper())

    rcd = RentCastData.open_db(db_folder='./Data', city=city, state=state)
    df = rcd.parse_sale_date2(rcd.data_processed)
    
    check1 = df['squareFootage'] > 1000
    check2 = df['squareFootage'] < 2000
    df_temp  = df[check1 & check2]
    # df_temp = df

    x = df_temp['months'].values / 12
    y = df_temp['price_per_sqft'].values

    # Avererages
    grouped_df = df_temp.groupby('months')['price_per_sqft'].mean().reset_index()
    x_avg = grouped_df['months'].values / 12
    y_avg = grouped_df['price_per_sqft'].values

    # y_average = df.average

    # plt.plot(x, y, lw=0, ms=2, marker='o', alpha=0.2)
    

    if filter:
        y_filt = sos_filter(y_avg, 1, 0.03, highlow='low', order=1)
        plt.semilogy(x_avg, y_filt, lw=2, ms=0, marker='o', alpha=0.8, label=label)
    else:
        plt.semilogy(x_avg, y_avg, lw=1, ms=0, marker='o', alpha=0.8, label=label)

    plt.ylim([50, 3000])

    # plt.xlim([40, 60])

    # x labels
    int_labels = np.arange(0, 60, 2.5)
    n = len(int_labels)
    lsd = df['lastSaleDate'].values[0]
    dt = datetime.datetime.strptime(lsd, '%Y-%m-%dT%H:%M:%S.%fZ')
    dt_labels = [rcd.add_months(dt, x * 30) for x in range(n)]
    dt_labels = [dt_label.strftime('%m/%Y') for dt_label in dt_labels]
    plt.xticks(int_labels, dt_labels, rotation=45)
    plt.yticks([10, 30, 100, 300, 1000], [10, 30, 100, 300, 1000])

    plt.title('Price per SQFT')
    plt.ylabel('Price per SQFT ($)')
    plt.xlabel('Time')
    plt.grid(True)

    # if index == 5:
    #     break

plt.legend(loc='best', ncol=2)

In [ ]:
filter = True

for table_names in [cities1, cities2]:

    plt.figure(figsize=(10, 4))

    for index, table_name in enumerate(table_names):

        table_name_split = table_name.split('_')
        state = table_name_split[-1]
        city = " ".join(table_name_split[:-1])
        label = '%s - %s' % (city.title(), state.upper())

        rcd = RentCastData.open_db(db_folder='./Data', city=city, state=state)
        df = rcd.parse_sale_date2(rcd.data_processed)
        
        check1 = df['squareFootage'] > 1000
        check2 = df['squareFootage'] < 2000
        df_temp  = df[check1 & check2]
        # df_temp = df

        x = df_temp['months'].values / 12
        y = df_temp['price_per_sqft'].values

        # Avererages
        grouped_df = df_temp.groupby('months')['price_per_sqft'].mean().reset_index()
        x_avg = grouped_df['months'].values / 12
        y_avg = grouped_df['price_per_sqft'].values
        
        # Filter and plot results
        if filter:
            y_filt = sos_filter(y_avg, 1, 0.03, highlow='low', order=1)
            plt.semilogy(x_avg, y_filt, lw=2, ms=0, marker='o', alpha=0.8, label=label)
        else:
            plt.semilogy(x_avg, y_avg, lw=1, ms=0, marker='o', alpha=0.8, label=label)

        plt.ylim([50, 3000])

        # x labels
        int_labels = np.arange(0, 60, 2.5)
        n = len(int_labels)
        lsd = df['lastSaleDate'].values[0]
        dt = datetime.datetime.strptime(lsd, '%Y-%m-%dT%H:%M:%S.%fZ')
        dt_labels = [rcd.add_months(dt, x * 30) for x in range(n)]
        dt_labels = [dt_label.strftime('%m/%Y') for dt_label in dt_labels]
        plt.xticks(int_labels, dt_labels, rotation=45)
        plt.yticks([10, 30, 100, 300, 1000], [10, 30, 100, 300, 1000])

        plt.title('Price per SQFT')
        plt.ylabel('Price per SQFT ($)')
        plt.xlabel('Time')
        plt.grid(True)

    plt.legend(loc='best', ncol=2)